# Email Auto Fill
- **Text Preprocessing**
- **Probabilistic Language Models**
    - *Unigram, Bigrams, Trigrams, N-grams*

In [1]:
import numpy as np
import pandas as pd
import nltk, re, string, contractions
from nltk.tokenize import sent_tokenize, word_tokenize

In [8]:
raw_data = pd.read_csv(r'F:\Muthu_2023\Personal\NextStep\NLP\NLP\Dataset\Email\email_truncated.csv')